In [1]:
import argparse
import numpy as np
import os
from math import log10, floor
from datetime import datetime
from glob import glob
import stat
import pickle
import math
import array
import scipy
from multiprocessing import Pool
from itertools import repeat
from pathlib import Path
from scipy.signal import correlate
from scipy.fft import ifft, fft, next_fast_len
import qetpy as qp
import matplotlib
import matplotlib.pyplot as plt
import pytesdaq.io.hdf5 as h5io
import pytesdaq
from detprocess.core import OptimumFilterTrigger, EventBuilder, Template, Noise, Salting
from detprocess import FilterData
from pprint import pprint
from scipy import stats, signal, interpolate, special, integrate
# ignore warnings
import warnings
warnings.filterwarnings('ignore')
import pandas as pd
import types
import mendeleev
import pickle as pkl
import sys
from darklim import constants

<frozen importlib._bootstrap>:241: RuntimeWarning: numpy.ndarray size changed, may indicate binary incompatibility. Expected 80 from C header, got 96 from PyObject


In [2]:
salt = Salting(verbose=True)

In [3]:
# continuous data directory, use argument "series" for list of series within continuous data directory
continuous_data = '/data/ucb/pylelab/run47/raw/continuous_I2_D20240712_T205330'
series = None
# 1.  Generate randoms
#  use either "random_rate" or nevents
#  you can specify minimum separation between randoms (default=100msec) and from/to the edge of 
# the continuous data
salt.generate_randoms(continuous_data, series=series, nevents=1000, min_separation_msec=100, ncores=1)

INFO: Found total of 54 raw data file(s) from 6 different series number!
INFO: Found total of 54 files from 6 different series number!
INFO Node #1: Acquiring randoms for series I2_D20240712_T205736
INFO Node #1: Acquiring randoms for series I2_D20240712_T213156
INFO Node #1: Acquiring randoms for series I2_D20240712_T220616
INFO Node #1: Acquiring randoms for series I2_D20240712_T224034
INFO Node #1: Acquiring randoms for series I2_D20240712_T231454
INFO Node #1: Acquiring randoms for series I2_D20240712_T234916
INFO: Randoms acquisition done!


In [4]:
salt._dataframe

#,series_number,event_number,dump_number,event_time,series_start_time,group_start_time,fridge_run_start_time,fridge_run_number,trigger_index,trigger_time,trigger_type,data_type,group_name,processing_id,trigger_prod_id,trigger_prod_group_name
0,220240712205736,100001,1,1720843074,19,264,28674,47,4547314,7.2757024,3,Test,continuous_I2_D20240712_T205330,nan,1,nan
1,220240712205736,100002,1,1720843084,29,274,28684,47,4148946,6.6383136,3,Test,continuous_I2_D20240712_T205330,nan,2,nan
2,220240712205736,100003,1,1720843094,39,284,28694,47,4502617,7.2041872,3,Test,continuous_I2_D20240712_T205330,nan,3,nan
3,220240712205736,100004,1,1720843099,44,289,28699,47,1415418,2.2646688,3,Test,continuous_I2_D20240712_T205330,nan,4,nan
4,220240712205736,100006,1,1720843118,63,308,28718,47,471838,0.7549408,3,Test,continuous_I2_D20240712_T205330,nan,5,nan
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
973,220240712234916,900008,9,1720855122,1767,12312,40722,47,2613962,4.1823392,3,Test,continuous_I2_D20240712_T205330,nan,159,nan
974,220240712234916,900009,9,1720855131,1776,12321,40731,47,1886633,3.0186128,3,Test,continuous_I2_D20240712_T205330,nan,160,nan
975,220240712234916,900010,9,1720855142,1787,12332,40742,47,2335294,3.7364704,3,Test,continuous_I2_D20240712_T205330,nan,161,nan
976,220240712234916,900011,9,1720855153,1798,12343,40753,47,3201968,5.1231488,3,Test,continuous_I2_D20240712_T205330,nan,162,nan


In [5]:
#channel_name = 'Mv3025pcRegular'
#ivsweep_file = '/data/ucb/pylelab/run46/filterdata/iv_didv_I2_D20240626_T202802/ivsweep_analysis_D20240918_T155128.hdf5'

#salt.set_iv_didv_results_from_file("/data/ucb/pylelab/run46/filterdata/iv_didv_I2_D20240626_T171500/ivsweep_analysis_D20240702_T123724.hdf5")

#salt.set_iv_didv_results_from_dict(channel="Mv3025pcBigFins",didv_results="/data/ucb/pylelab/run46/processed/didvs/didvresult_Mv3025pcBigFins_0624")
#filterdataIO = FilterData()
#filterdataIO.load_hdf5("/data/ucb/pylelab/run46/filterdata/iv_didv_I2_D20240626_T171500/ivsweep_analysis_D20240702_T123724.hdf5")
#filterdataIO.describe()

In [6]:
salt.set_iv_didv_results_from_file("/data/ucb/pylelab/run46/filterdata/continuous_I2_D20240622_T142759/didv_analysis_D20240919_T093247.hdf5")

INFO: Loading filter data from file /data/ucb/pylelab/run46/filterdata/continuous_I2_D20240622_T142759/didv_analysis_D20240919_T093247.hdf5


In [7]:
foo = salt._ivdidv_data

In [8]:
foo

{'Mv3025pcRegular': {'smallsignalparams': {'rsh': 0.004999999888241291,
   'rp': 0.002620722961208931,
   'r0': 0.09302457160217095,
   'beta': 1.1699497009463706,
   'l': 28.41537782620757,
   'L': 2.463427344731654e-07,
   'tau0': 0.0007124910605162721,
   'gratio': nan,
   'tau3': nan,
   'dt': 2.2050123541979865e-06},
  'biasparams': {'rp': 0.002620722961208931,
   'rp_err': 5.241445922417862e-05,
   'rn': 0.8865034029293155,
   'rn_err': 0.017882559330361102,
   'rshunt': 0.004999999888241291,
   'rshunt_err': 9.999999776482583e-05,
   'rsh': 0.004999999888241291,
   'rsh_err': 9.999999776482583e-05,
   'i0_off': -6.602635320797366e-08,
   'i0_off_err': 1.0817279007210398e-11,
   'ibias_off': -6.774514183834787e-10,
   'ibias_off_err': 1.8188998233804914e-11,
   'lgc_invert_offset': True,
   'close_loop_norm': 625000.0,
   'output_variable_offset': -0.12087912112474442,
   'output_variable_gain': 50.0,
   'group_name_sweep': 'iv_didv_I2_D20240626_T202802',
   'i0_variable_offset':

In [9]:
blah = salt.get_ivsweep_results('Mv3025pcRegular')

In [10]:
blah

{'rp': 0.002620722961208931,
 'rp_err': 5.241445922417862e-05,
 'rn': 0.8865034029293155,
 'rn_err': 0.017882559330361102,
 'rshunt': 0.004999999888241291,
 'rshunt_err': 9.999999776482583e-05,
 'rsh': 0.004999999888241291,
 'rsh_err': 9.999999776482583e-05,
 'i0_off': -6.602635320797366e-08,
 'i0_off_err': 1.0817279007210398e-11,
 'ibias_off': -6.774514183834787e-10,
 'ibias_off_err': 1.8188998233804914e-11,
 'lgc_invert_offset': True,
 'close_loop_norm': 625000.0,
 'output_variable_offset': -0.12087912112474442,
 'output_variable_gain': 50.0,
 'group_name_sweep': 'iv_didv_I2_D20240626_T202802',
 'i0_variable_offset': -9.670329689979553e-06,
 'sc_didv_fit_L': 5.295518636880649e-07,
 'sc_didv_fit_rp': 0.002634684127221532,
 'sc_didv_fit_dt': 1.3117065008168692e-06,
 'normal_didv_fit_L': 6.747521463797285e-07,
 'normal_didv_fit_rn': 0.8754639140461822,
 'normal_didv_fit_dt': 6.888104176782892e-07,
 'tes_bias': 8.012827838827832e-06,
 'ibias': 8.013505290246215e-06,
 'ibias_err': 1.81889

In [11]:
didv = salt.get_didv_results("Mv3025pcRegular",poles=2)

In [12]:
didv

{'lgcfix': None,
 'params_array': array([ 2.09479364e-01, -3.05639337e-01,  0.00000000e+00, -2.59887376e-05,
         1.17597614e-06,  0.00000000e+00,  2.20501235e-06]),
 'cov': array([[ 5.23016999e-06, -5.19092603e-06,  0.00000000e+00,
          4.56184037e-10, -8.77461462e-12,  0.00000000e+00,
          5.80026398e-12],
        [-5.19092602e-06,  5.19926538e-06,  0.00000000e+00,
         -4.48754857e-10,  8.43446106e-12,  0.00000000e+00,
         -5.67543922e-12],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00],
        [ 4.56184037e-10, -4.48754857e-10,  0.00000000e+00,
          8.37005822e-14, -1.34813203e-15,  0.00000000e+00,
          1.37089972e-15],
        [-8.77461463e-12,  8.43446108e-12,  0.00000000e+00,
         -1.34813203e-15,  2.35023372e-15,  0.00000000e+00,
         -8.89720123e-16],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
          0.00000000e+00,  0

In [ ]:
ivdidv_df = filterdataIO.get_ivsweep_data("Mv3025pcRegular")

In [7]:
ivresults = filterdataIO.get_ivsweep_results("Mv3025pcRegular")

In [ ]:
ivdidv_df

In [ ]:
didvdf = filterdataIO.get_didv_dataframe("Mv3025pcBigFins")